## PurpleAir Data
Data are available via PurpleAir's API. You will need to use a gmail account and create an API key via [this dashboard](https://develop.purpleair.com/sign-in?redirectURL=%2Fdashboards%2Fkeys). You should create a "Read" key that has a status of "Enabled". It's a good idea to add a label, host restrictions restrict the use of the key to certain machines, you do not need to set these. 

TODO: insert screen shot image

Once you have generated your key, you can "read" the key value to use in making requests. First, run the cell below and enter the key you generated when prompted. 

In [1]:
import getpass

api_key = getpass.getpass("Enter your API key: ")

Test your API key by running the code below. It should show the message "Key submit was successful" if your key is valid. 

In [2]:
import requests

url = "https://api.purpleair.com/v1/keys"

headers = {
    "X-API-Key": api_key 
}

response = requests.get(url, headers=headers)

if response.status_code == 201:
    print('Key submit was successful')
    data = response.json()
    print(data)
else:
    print(f"Request failed with status code: {response.status_code}")

Key submit was successful
{'api_version': 'V1.0.14-0.0.58', 'time_stamp': 1741284515, 'api_key_type': 'READ'}


If the API key is valid, a bounding box can be used to search for sensors. The coordinates in the cell below represent the longitude and latitude of the northwest and southeast corners of a box that encloses the South Bronx. The API request returns the identifiers of sensors within that bounding box. 

In [32]:
# corner latitude and longitudes in decimal degrees
nwlat = 40.9
nwlng = -73.95
selat = 40.80
selng = -73.78

url = 'https://api.purpleair.com/v1/sensors'

headers = {"X-API-Key": api_key}

params = {
    'fields':'name,latitude,longitude,position_rating,last_seen',
    'location_type':0,
    'nwlng':nwlng,
    'nwlat':nwlat,
    'selng':selng,
    'selat':selat

}

with requests.get(url, headers=headers, params=params) as response:

    if response.status_code == 200:
        print('Success')
        data = response.json()
        print(data)
    else:
        print(f"Request failed with status code: {response.status_code}")


Success
{'api_version': 'V1.0.14-0.0.58', 'time_stamp': 1741284746, 'data_time_stamp': 1741284737, 'location_type': 0, 'max_age': 604800, 'firmware_default_version': '7.02', 'fields': ['sensor_index', 'last_seen', 'name', 'position_rating', 'latitude', 'longitude'], 'data': [[90249, 1741284632, 'FreshAir-O4', 3, 40.861225, -73.89016], [90283, 1741284653, 'SIS-roof', 5, 40.81536, -73.888374], [90389, 1741284356, 'FA-AHo', 5, 40.83022, -73.92234], [91423, 1741284717, 'FA-O2b', 5, 40.861134, -73.891556], [91899, 1741283900, 'FA-O7', 5, 40.83016, -73.9219], [92169, 1741284727, 'FA-CTKo', 5, 40.837517, -73.91544], [92171, 1741284684, 'FreshAir-O1', 5, 40.860455, -73.88581], [172111, 1741284723, 'Bronx Center for Science and Mathematics - outdoor', 5, 40.833763, -73.902565], [178411, 1741284724, 'WE ACT for Environmental Justice Office', 0, 40.829655, -73.94472], [183603, 1741284714, 'Hudson View Gardens', 5, 40.853676, -73.938484], [188617, 1741284686, 'Spuyten Duyvil: Arlington at 227', 0,

In [33]:
len(data['data'])

17

In [34]:
data['data'][0]

[90249, 1741284632, 'FreshAir-O4', 3, 40.861225, -73.89016]

In [35]:
import pandas as pd
import geopandas as gpd

In [36]:
df = pd.DataFrame(data['data'], columns=data['fields'])
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.longitude, df.latitude), 
    crs="EPSG:4326"
)

print(gdf.shape)

(17, 7)


In [31]:
import matplotlib.pyplot as plt
import folium
# map to show the sensors
# TODO: expand bounding box
gdf.explore()
